In [1]:
import pandas as pd
import nltk 
import langid
import re
import nltk

In [2]:
def add_to_dic(dic,df,sheet):
    str1='' 
#FIND ENGLISH AND JOIN INTO DICTIONATY for each sheet
    print(f"processsing sheet{sheet}.")
    for i in df.text.values: 
        lang = langid.classify(str(i))
        if lang and 'en'in lang[0]:
            str1 = str1 +'\n'+ str(i)           
            dic[sheet]= str1
    return dic
    

In [3]:
excel = pd.ExcelFile('28130006.xlsx') # opening excel fiile
d={} # declaring an empty dict
for sheet in excel.sheet_names: # iterate over each sheets
    df = excel.parse(sheet)
    df = df.dropna(axis=1,how='all')
    df = df.dropna(axis=0,how='all')
    df.reset_index(inplace=True,drop=True)
    if df.iloc[0][0]== "text":
      
        new_header = df.iloc[0] #grab the first row for the header
        df = df[1:] #take the data less the header row
        df.columns = new_header #set the header row as the df header
        df.head()
        d=add_to_dic(d,df,sheet)

    else:
        
        d=add_to_dic(d,df,sheet)

processsing sheet2020-03-22.
processsing sheet2020-03-23.
processsing sheet2020-03-24.
processsing sheet2020-03-25.
processsing sheet2020-03-26.
processsing sheet2020-03-27.
processsing sheet2020-03-28.
processsing sheet2020-03-29.
processsing sheet2020-03-30.
processsing sheet2020-03-31.
processsing sheet2020-04-01.
processsing sheet2020-04-02.
processsing sheet2020-04-03.
processsing sheet2020-04-04.
processsing sheet2020-04-05.
processsing sheet2020-04-06.
processsing sheet2020-04-07.
processsing sheet2020-04-08.
processsing sheet2020-04-09.
processsing sheet2020-04-10.
processsing sheet2020-04-11.
processsing sheet2020-04-12.
processsing sheet2020-04-13.
processsing sheet2020-04-14.
processsing sheet2020-04-15.
processsing sheet2020-04-16.
processsing sheet2020-04-17.
processsing sheet2020-04-18.
processsing sheet2020-04-19.
processsing sheet2020-04-20.
processsing sheet2020-04-21.
processsing sheet2020-04-22.
processsing sheet2020-04-23.
processsing sheet2020-04-24.
processsing sh

In [4]:
#Tokenizer the Data Dictionary
from nltk.tokenize import RegexpTokenizer
#FIND WORDS FROM SENTENCES- SEPERATE WORDS AND MAKE LOWERCASE
tokenizer = RegexpTokenizer("[a-zA-Z]+(?:[-'][a-zA-Z]+)?")

def tokenizedata(sheet): #from Tut05
    """
        the tokenization function is used to tokenize raw data in Dictionary
    """
    raw_data = d[sheet].lower() 
    unigram_tokens = tokenizer.tokenize(raw_data)
    return (sheet, unigram_tokens)

##First Dictionary with all the tokens for each Day
data_tokenized = dict(tokenizedata(sheet) for sheet in d.keys())
                      

In [5]:
len(data_tokenized.keys()) #to check if all the sheets have been captured i.e 81

81

In [6]:
#Creating the Dict for most Frequent 100 bigrams for each day
from nltk.util import ngrams
from nltk.probability import *
collect_bi={}
for keys,values in data_tokenized.items():
    bi=[]
    for i in ngrams(values,2):
        bi.append(i)
    fdbigram = FreqDist(bi)
    collect_bi[keys]=fdbigram.most_common(100)


In [7]:
#saving the Dict of 100 Bigrams to file
save_file = open("arushi_100bigrams.txt", 'w')
for k,v, in collect_bi.items():
    save_file.write("%s:%s\n"%(k,v))
  
save_file.close()

In [8]:
#from Tut05----- start cleaning up and proceed with Vocab
from __future__ import division
from itertools import chain
import itertools
all_words = list(chain.from_iterable(data_tokenized.values()))
#Make a Voc
all_vocab = set(all_words)
lexical_diversity = len(all_words)/len(all_vocab)
print ("Vocabulary size: ",len(all_vocab),"\nTotal number of tokens: ", len(all_words), \
"\nLexical diversity: ", lexical_diversity)

Vocabulary size:  199616 
Total number of tokens:  2306778 
Lexical diversity:  11.55607766912472


In [9]:
#This is to check the top most words for the whole Excel file
fd_1 = FreqDist(all_words)
fd_1.most_common(10)

[('t', 91617),
 ('co', 86730),
 ('https', 86522),
 ('the', 70496),
 ('to', 51822),
 ('covid', 41860),
 ('of', 36085),
 ('coronavirus', 31317),
 ('and', 30969),
 ('in', 29697)]

In [10]:
#for Document Frequency to find out Rare Tokens and 
words_2 = list(chain.from_iterable([set(value) for value in data_tokenized.values()]))
fd_2 = FreqDist(words_2)
fd_2.most_common(10)

[('around', 81),
 ('important', 81),
 ('hospital', 81),
 ('together', 81),
 ('best', 81),
 ('fight', 81),
 ("let's", 81),
 ('long', 81),
 ('action', 81),
 ('like', 81)]

In [11]:
stopwords = []
with open('./stopwords_en.txt') as f:
    stopwords = f.read().splitlines()

In [12]:
#
data_tokenized_1 = {}
stopwordsSet = set(stopwords)
for sheet in data_tokenized.keys():
    data_tokenized_1[sheet] = [w for w in data_tokenized[sheet] if w not in stopwordsSet]
    
#data_tokenized = dict(tokenizedata(sheet) for sheet in d.keys())

In [13]:
len(data_tokenized_1.keys())

81

In [14]:
###Lets create chain of words (bags of words) after StopWords removed Word3
words_3 = list(chain.from_iterable(data_tokenized_1.values()))
fd_3 = FreqDist(words_3)
list(all_vocab - set(fd_3.keys()))


['six',
 'is',
 'until',
 'liked',
 "haven't",
 'me',
 'again',
 'around',
 "weren't",
 'might',
 'done',
 'indeed',
 'hi',
 'doing',
 'lest',
 'g',
 'us',
 'certain',
 'someone',
 'uses',
 'being',
 'if',
 'none',
 'ltd',
 'different',
 'knows',
 'merely',
 'should',
 'just',
 'would',
 'seriously',
 'it',
 "aren't",
 "you've",
 'herself',
 'several',
 'often',
 'inc',
 'former',
 'greetings',
 'go',
 'concerning',
 'become',
 'since',
 'less',
 'most',
 'your',
 'up',
 'downwards',
 'others',
 'thank',
 'inward',
 'f',
 'come',
 'yes',
 'saying',
 'yet',
 'overall',
 'que',
 'under',
 'actually',
 'see',
 'etc',
 'ask',
 'somehow',
 'seeming',
 'aside',
 'vs',
 'hence',
 'below',
 'while',
 'when',
 'seeing',
 "we've",
 'otherwise',
 'always',
 'ignored',
 'himself',
 'thanx',
 'nobody',
 'together',
 'among',
 'latter',
 'which',
 'against',
 'particular',
 'thoroughly',
 'toward',
 'indicated',
 'seem',
 'sup',
 "wasn't",
 'associated',
 'be',
 'respectively',
 'wherein',
 'specifi

In [15]:
#This is to check the top most words_3 which are after stop words
fd_1 = FreqDist(words_3)
fd_1.most_common(10)

[('https', 86522),
 ('covid', 41860),
 ('coronavirus', 31317),
 ('people', 6638),
 ('amp', 6494),
 ('pandemic', 5505),
 ('cases', 5110),
 ('news', 3869),
 ('lockdown', 3845),
 ('trump', 3663)]

In [16]:
###Check the document Freq after stop words are removed
words_4 = list(chain.from_iterable([set(value) for value in data_tokenized_1.values()]))
fd_2 = FreqDist(words_4)
fd_2.most_common(10)

[('important', 81),
 ('hospital', 81),
 ('fight', 81),
 ('long', 81),
 ('action', 81),
 ('covid', 81),
 ('love', 81),
 ('morning', 81),
 ('disease', 81),
 ('years', 81)]

In [18]:
###we need to remove the Rare tokens (5days) and Context_independent /Dependent (60 days)
lessFreqWords = [k for k, v in fd_2.items() if v > 4 and v < 60 ]
len(lessFreqWords)

15705

In [20]:
lessFreqWords=set(lessFreqWords)

In [21]:

#data_tokenized_2 = {}
def removeLessFreqWords(sheet):
    return (sheet, [w for w in data_tokenized_1[sheet] if w in lessFreqWords])

data_tokenized_2 = dict(removeLessFreqWords(sheet) for sheet in data_tokenized_1.keys())


In [22]:
len(data_tokenized_2.keys())

81

In [23]:
###new Chain of Words after removing less freq/Rare
words_5 = list(chain.from_iterable(data_tokenized_2.values()))
fd_1 = FreqDist(words_5)
fd_1.most_common(10)

[('protests', 355),
 ('sunday', 252),
 ('cummings', 232),
 ('tuesday', 188),
 ('alert', 179),
 ('zealand', 179),
 ('migrant', 162),
 ('riots', 157),
 ('george', 154),
 ('matthancock', 152)]

In [24]:
words_6 = list(chain.from_iterable([set(value) for value in data_tokenized_2.values()]))
fd_2 = FreqDist(words_6)
fd_2.most_common(10)

[('map', 59),
 ('lets', 59),
 ('offers', 59),
 ('games', 59),
 ('homeless', 59),
 ('speaking', 59),
 ('shares', 59),
 ('highly', 59),
 ('scared', 59),
 ('fault', 59)]

In [25]:
#check after rare tokens removed
all_vocab_1 = set(words_5)
lexical_diversity = len(words_5)/len(all_vocab_1)
print ("Vocabulary size: ",len(all_vocab_1),"\nTotal number of tokens: ", len(words_5), \
"\nLexical diversity: ", lexical_diversity)

Vocabulary size:  15705 
Total number of tokens:  350404 
Lexical diversity:  22.311620503024514


In [26]:
#lets remove less than 3 char words
data_tokenized_3={}
for sheet in data_tokenized_2.keys():
    data_tokenized_3[sheet] = [w for w in data_tokenized_2[sheet] if len(w) >3]

In [27]:
len(data_tokenized_3.keys())

81

In [28]:
###new Chain of Words after removing less freq/Rare
words_7 = list(chain.from_iterable(data_tokenized_3.values()))
fd_1 = FreqDist(words_7)
fd_1.most_common(10)

[('protests', 355),
 ('sunday', 252),
 ('cummings', 232),
 ('tuesday', 188),
 ('alert', 179),
 ('zealand', 179),
 ('migrant', 162),
 ('riots', 157),
 ('george', 154),
 ('matthancock', 152)]

In [29]:
words_8 = list(chain.from_iterable([set(value) for value in data_tokenized_3.values()]))
fd_2 = FreqDist(words_8)
fd_2.most_common(10)

[('lets', 59),
 ('offers', 59),
 ('games', 59),
 ('homeless', 59),
 ('speaking', 59),
 ('shares', 59),
 ('highly', 59),
 ('scared', 59),
 ('fault', 59),
 ('immediately', 59)]

In [30]:
all_vocab_2 = set(words_7)
lexical_diversity = len(words_7)/len(all_vocab_2)
print ("Vocabulary size: ",len(all_vocab_2),"\nTotal number of tokens: ", len(words_7), \
"\nLexical diversity: ", lexical_diversity)

Vocabulary size:  13636 
Total number of tokens:  294941 
Lexical diversity:  21.629583455558816


In [31]:
###Lets do Stemming
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

data_tokenized_4={}
for sheet in data_tokenized_3.keys():
    data_tokenized_4[sheet] = [stemmer.stem(w) for w in data_tokenized_3[sheet]]

In [32]:
len(data_tokenized_4.keys())

81

In [33]:
words_9 = list(chain.from_iterable(data_tokenized_4.values()))
fd_1 = FreqDist(words_9)
fd_1.most_common(10)

[('protest', 606),
 ('oper', 401),
 ('lift', 353),
 ('donat', 351),
 ('delay', 330),
 ('suggest', 309),
 ('predict', 307),
 ('celebr', 297),
 ('migrant', 293),
 ('save', 289)]

In [34]:
words_10 = list(chain.from_iterable([set(value) for value in data_tokenized_4.values()]))
fd_2 = FreqDist(words_10)
fd_2.most_common(10)

[('celebr', 80),
 ('donat', 80),
 ('lift', 79),
 ('oper', 79),
 ('surviv', 79),
 ('save', 79),
 ('walk', 79),
 ('urg', 78),
 ('support', 78),
 ('design', 78)]

In [35]:
all_vocab_3 = set(words_9)
lexical_diversity = len(words_9)/len(all_vocab_3)
print ("Vocabulary size: ",len(all_vocab_3),"\nTotal number of tokens: ", len(words_9), \
"\nLexical diversity: ", lexical_diversity)

Vocabulary size:  9799 
Total number of tokens:  294941 
Lexical diversity:  30.099091744055517


In [36]:
####Create Unigrams here from data_tokenized_4
#####
collect_uni={}
for keys,values in data_tokenized_4.items():
    uni=[]
    for i in values:
        uni.append(i)
    unigram = FreqDist(uni)
    collect_uni[keys]=unigram.most_common(100)

In [37]:
save_file = open("arushi_100unigrams.txt", 'w')
for k,v in collect_uni.items():
    save_file.write("%s:%s\n"%(k,v))
  
save_file.close()

In [38]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bigram_finder = nltk.collocations.BigramCollocationFinder.from_words(words_9)
bigram_finder.apply_word_filter(lambda w: len(w) < 3)# or w.lower() in ignored_words)
top_200_bigrams = bigram_finder.nbest(bigram_measures.pmi, 200) # Top-100 bigrams
top_200_bigrams

[('dlaignit', 'socialsel'),
 ('nutan', 'jyot'),
 ('socialsel', 'digitalsel'),
 ('tradingeconom', 'macrocalendar'),
 ('asitha', 'jayawardena'),
 ('kmnw', 'igf'),
 ('macrocalendar', 'economiccalendar'),
 ('roboinfo', 'robotex'),
 ('sperm', 'smoothi'),
 ('artcal', 'artistcal'),
 ('artistcal', 'artistscal'),
 ('artistscal', 'artpromot'),
 ('artpromot', 'coronavirusart'),
 ('meaningfulgrowth', 'geniouxmg'),
 ('nagendra', 'bandi'),
 ('nirmala', 'sitharaman'),
 ('sierra', 'leon'),
 ('tejasvi', 'surya'),
 ('foryoupag', 'foryou'),
 ('agenparl', 'covidagenparl'),
 ('apolloquiboloy', 'kingdomofjesuschrist'),
 ("do'", "don't"),
 ('kingdomofjesuschrist', 'bebless'),
 ('lauri', 'garrett'),
 ('logoanim', 'madeonfiverr'),
 ('mdncvc', 'ruhyuybt'),
 ('wbzpq', 'hvbj'),
 ('wltxnlcv', 'prnyztb'),
 ('acquit', 'maniac'),
 ('maryam', 'rajavi'),
 ('morninglivesabc', 'leannemana'),
 ('angi', 'motshekga'),
 ('pradhan', 'mantri'),
 ('lockdowninsa', 'lockdowninsouthafrica'),
 ('nova', 'scotia'),
 ('funk', 'akindel

In [39]:
from nltk.tokenize import MWETokenizer
mwetokenizer = MWETokenizer(top_200_bigrams)
data_tokenized_5=  dict((keys, mwetokenizer.tokenize(values)) for keys,values in data_tokenized_4.items())
words_11 = list(chain.from_iterable(data_tokenized_5.values()))
colloc_voc = set(words_11)
print(len(list(colloc_voc)))

9917


In [40]:
for key in data_tokenized_4.keys():
    diff = set(data_tokenized_5[key])-set(data_tokenized_4[key])
    if len(diff) != 0:
        print (key, diff)

2020-03-22 {'janeruth_aceng', 'paulo_dybala'}
2020-03-23 {'funk_akindel', 'tcot_ccot', 'uddhav_thackeray', 'skinnergj_crnew'}
2020-03-24 {'adhanom_ghebreyesu', 'nirmala_sitharaman', 'sierra_leon', 'whereisbori_torygenocid', 'sardanarohit_anjanaomkashyap', 'geonew_urdu', 'staythefhom_socialdistancingnow'}
2020-03-25 {'koko_pimentel', 'lauri_garrett', 'tasmania_polita', 'skinnergj_crnew', 'uddhav_thackeray', 'babajid_sanwo-olu', 'wltxnlcv_prnyztb', "do'_don't"}
2020-03-26 {'adhanom_ghebreyesu', 'tradingeconom_macrocalendar', 'rishi_sunak', 'infrar_thermomet', 'lqmh_kart', 'janeruth_aceng', 'tejasvi_surya', 'economiccalendar_unitedkingdom', 'jerri_falwel', 'kyli_jenner', 'nach_mdncvc', 'capt_amarind'}
2020-03-27 {'dayslockdownsa_stayhomesa', 'koko_pimentel', 'hippocrat_oath', 'skinnergj_crnew', 'covidagenparl_iorestoacasa', 'southafricalockdown_lockdowninsa', 'craig_spencer', 'hipaa_compliant'}
2020-03-28 {'dayslockdownsa_stayhomesa', 'nigel_farag', 'rishi_sunak', 'ptioffici_pmln', 'trish

In [41]:
sheets = []
bag_of_words = []
for sheet, tokens in data_tokenized_5.items():
    sheets.append(sheet)
    txt = ' '.join(tokens)
    bag_of_words.append(txt)

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(input = 'content',analyzer = "word")


In [43]:
data_features = vectorizer.fit_transform(bag_of_words)
print (data_features.shape)

(81, 9794)


In [44]:
full_vocab = vectorizer.get_feature_names()
len(list(full_vocab))

9794

In [45]:
######Write Vocab to fie and creating a Dict for VOCAB
full_vocab=set(full_vocab)
vocab_dict = {}
full_vocab=sorted(full_vocab)
save_file = open("arushi_Vocab.txt", 'w')
for count, item in enumerate(full_vocab):
    #print(item, count)
    vocab_dict[item] = item

    save_file.write("%s:%s\n"%(item,count))
  
save_file.close()

In [46]:
save_file = open("arushi_countvec.txt", 'w')
vocab3=vectorizer.get_feature_names()
cx = data_features.tocoo() # return the coordinate representation of a sparse matrix
counter=0
save_file.write(sheets[counter])
for i,j,v in itertools.zip_longest(cx.row, cx.col, cx.data):
    if i==counter:
        save_file.write(',' + str(j) + ':' + str(v))
    else:
        counter+=1
        save_file.write('\n'+ sheets[counter])
    
save_file.close()